In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aneesarom/rider-with-helmet-without-helmet-number-plate")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'rider-with-helmet-without-helmet-number-plate' dataset.
Path to dataset files: /kaggle/input/rider-with-helmet-without-helmet-number-plate


In [4]:
!pip install ultralytics --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00


In [5]:
import os
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
data_dir = "/kaggle/input/rider-with-helmet-without-helmet-number-plate"

In [7]:
train_dir = "/kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images"
val_dir = "/kaggle/input/rider-with-helmet-without-helmet-number-plate/val/images"

In [10]:
classes_path = "/kaggle/input/rider-with-helmet-without-helmet-number-plate/classes.txt"
with open(classes_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]


In [11]:
num_classes = len(class_names)
num_classes

4

In [12]:
yaml_content = f"""
path = {data_dir} ,
train = {train_dir} ,
valid = {val_dir} ,
num_classes = {num_classes} ,
names = {class_names}

"""


In [16]:
import os

os.makedirs("/kaggle/working", exist_ok=True)

print("working is exist ? ", os.path.exists("/kaggle/working"))

yaml_content = """
train: /kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images
val: /kaggle/input/rider-with-helmet-without-helmet-number-plate/val/images

nc: 4
names: ["with helmet", "without helmet", "rider", "number plate"]
"""
yaml_path = "/kaggle/working/data.yaml"

file = open(yaml_path, "w")
file.write(yaml_content)
file.close()

print("✅ yaml_file is finished with classes:", class_names)

working is exist ?  True
✅ yaml_file is finished with classes: ['with helmet', 'without helmet', 'rider', 'number plate']


In [17]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

In [19]:
results = model.train(
    data="/kaggle/working/data.yaml",
    epochs=50,
    batch=16,
    imgsz=640,
    project="yolo_train",
    name="helmet_model",
    device=0
)


Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=helmet_model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose

In [24]:
from ultralytics import YOLO

model = YOLO("/content/yolo_train/helmet_model2/weights/best.pt")

# test on photo
results = model.predict("/kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images/new100.jpg", show=True, conf=0.5)



image 1/1 /kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images/new100.jpg: 480x640 2 without helmets, 1 rider, 1 number plate, 8.3ms
Speed: 4.0ms preprocess, 8.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)


In [26]:
from ultralytics import YOLO

model = YOLO("/content/yolo_train/helmet_model2/weights/best.pt")

# test on vedio
results = model.predict(
    source="/content/helmet_video.mp4",
    show=True,
    conf=0.5,
    save=True,
    save_txt=False       )



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/643) /content/helmet_video.mp4: 384x640 (no detections), 10.0ms
video 1/1 (frame 2/643) /content/helmet_video.mp4: 384x640 (no detections), 7.5ms
video 1/1 (frame 3/643) /content/helmet_video.mp4: 384x640 (no detections), 8.0ms
video 1/1 (frame 4/643) /content/helmet_video.mp4: 384x640 (no detections), 7.0ms
video 1/1 (frame 5/643) /content/helmet_video.mp4: 384x640 (no detections), 7.5ms
video 1/1 (frame 6/643) /content/helmet_video

In [27]:
from ultralytics import YOLO

model = YOLO("/content/yolo_train/helmet_model2/weights/best.pt")

# test on vedio
results = model.predict(
    source="/content/bike-test.mp4",
    show=True,
    conf=0.5,
    save=True,
    save_txt=False       )



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/253) /content/bike-test.mp4: 544x640 2 riders, 50.4ms
video 1/1 (frame 2/253) /content/bike-test.mp4: 544x640 1 rider, 1 number plate, 13.8ms
video 1/1 (frame 3/253) /content/bike-test.mp4: 544x640 2 riders, 16.8ms
video 1/1 (frame 4/253) /content/bike-test.mp4: 544x640 1 with helmet, 2 riders, 1 number plate, 10.8ms
video 1/1 (frame 5/253) /content/bike-test.mp4: 544x640 1 with helmet, 2 riders, 7.6ms
video 1/1 (frame 6/253) /conten

In [28]:
from ultralytics import YOLO

model = YOLO("/content/yolo_train/helmet_model2/weights/best.pt")

# test on vedio
results = model.predict(
    source="/content/workers helmets.mp4",
    show=True,
    conf=0.5,
    save=True,
    save_txt=False       )



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/900) /content/workers helmets.mp4: 384x640 (no detections), 9.3ms
video 1/1 (frame 2/900) /content/workers helmets.mp4: 384x640 (no detections), 7.4ms
video 1/1 (frame 3/900) /content/workers helmets.mp4: 384x640 (no detections), 8.1ms
video 1/1 (frame 4/900) /content/workers helmets.mp4: 384x640 (no detections), 7.0ms
video 1/1 (frame 5/900) /content/workers helmets.mp4: 384x640 (no detections), 7.2ms
video 1/1 (frame 6/900) /conten